<a href="https://colab.research.google.com/github/bryanjiang1204/yes/blob/main/Video_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fvcore
!pip install av
!pip install pytorchvideo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20221221-py3-none-any.whl size=61400 sha256=5ea6e44343bd2f0d8e424aec2e8ff51d491f8b1e411413ca5907779c43c7b011
  Stored in directory: /root/.cache/pip/wheels/01/c0/af/77c1cf53a1be9e42a52b48e5af2169d40ec2e89f7362489dd0
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31530 sha256=e904de74c8251fe7a706d3a54bcd25ce64c5f2dccf956ab2e270a0e085610e44
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built fvcore iopath
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ...

In [ ]:
import torch

model_name = 'x3d_s'
model = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True)

device = "cpu"
model = model.eval()
model = model.to(device)

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [ ]:
import json
import urllib
from pytorchvideo.data.encoded_video import EncodedVideo

from torchvision.transforms import Compose, Lambda
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo,
)
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample
)

mean = [0.45, 0.45, 0.45]
std = [0.225, 0.225, 0.225]
frames_per_second = 30


transform_params = {
        "side_size": 182,
        "crop_size": 182,
        "num_frames": 13,
        "sampling_rate": 6,
    }

transform =  ApplyTransformToKey(
    key="video",
    transform=Compose(
        [
            UniformTemporalSubsample(transform_params["num_frames"]),
            Lambda(lambda x: x/255.0),
            NormalizeVideo(mean, std),
            ShortSideScale(size=transform_params["side_size"]),
            CenterCropVideo(
                crop_size=(transform_params["crop_size"], transform_params["crop_size"])
            )
        ]
    ),
)

clip_duration = (transform_params["num_frames"] * transform_params["sampling_rate"])/frames_per_second

In [ ]:
json_url = "https://dl.fbaipublicfiles.com/pyslowfast/dataset/class_names/kinetics_classnames.json"
json_filename = "kinetics_classnames.json"
try: urllib.URLopener().retrieve(json_url, json_filename)
except: urllib.request.urlretrieve(json_url, json_filename)

with open(json_filename, "r") as f:
    kinetics_classnames = json.load(f)

# Create an id to label name mapping
kinetics_id_to_classname = {}
for k, v in kinetics_classnames.items():
    kinetics_id_to_classname[v] = str(k).replace('"', "")

In [ ]:
url_link = "https://dl.fbaipublicfiles.com/pytorchvideo/projects/archery.mp4"
video_path = 'archery.mp4'
try: urllib.URLopener().retrieve(url_link, video_path)
except: urllib.request.urlretrieve(url_link, video_path)

In [ ]:
start_sec = 1
end_sec = start_sec + clip_duration

video = EncodedVideo.from_path(video_path)

video_data = video.get_clip(start_sec=start_sec, end_sec=end_sec)
video_data = transform(video_data)

inputs = video_data["video"]
inputs = inputs.to(device)

In [ ]:
print(model)

preds = model(inputs[None,...])

post_act = torch.nn.Softmax(dim=1)
preds = post_act(preds)
pred_classes = preds.topk(k=3).indices[0]

pred_class_names = [kinetics_id_to_classname[int(i)] for i in pred_classes]
print("Top predicted label: %s" % ", ".join(pred_class_names))

In [ ]:
import torch
from torchvision.models import resnet50
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.detection.mask_rcnn import MaskRCNN
from torchvision.models.detection.backbone_utils import LastLevelMaxPool
from torchvision.ops.feature_pyramid_network import FeaturePyramidNetwork

m = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True).eval()
train_nodes, eval_nodes = get_graph_node_names(m)

print(train_nodes)

class Resnet50WithFPN(torch.nn.Module):
    def __init__(self):
        super(Resnet50WithFPN, self).__init__()
        # Get a resnet50 backbone
        m = torch.hub.load('facebookresearch/pytorchvideo', model_name, pretrained=True).eval()

        # Extract 4 main layers (note: MaskRCNN needs this particular name
        # mapping for return nodes)
        self.body = create_feature_extractor(
            m, return_nodes={'blocks.5.proj': 'blank',})
        # Dry run to get number of channels for FPN
        inp = inputs[None,...]
        with torch.no_grad():
            out = self.body(inp)
        in_channels_list = [o.shape[1] for o in out.values()]
        # Build FPN


    def forward(self, x):
        x = self.body(x)
        return x

nu = Resnet50WithFPN()
prediction = nu(inputs[None,...])
prediction['blank'].shape


Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


['x', 'blocks.0.conv.conv_t', 'blocks.0.conv.conv_xy', 'blocks.0.norm', 'blocks.0.activation', 'blocks.1.res_blocks.0.branch1_conv', 'blocks.1.res_blocks.0.branch2.conv_a', 'blocks.1.res_blocks.0.branch2.norm_a', 'blocks.1.res_blocks.0.branch2.act_a', 'blocks.1.res_blocks.0.branch2.conv_b', 'blocks.1.res_blocks.0.branch2.norm_b.0', 'blocks.1.res_blocks.0.branch2.norm_b.1.mean', 'blocks.1.res_blocks.0.branch2.norm_b.1.block.0', 'blocks.1.res_blocks.0.branch2.norm_b.1.block.1', 'blocks.1.res_blocks.0.branch2.norm_b.1.block.2', 'blocks.1.res_blocks.0.branch2.norm_b.1.block.3', 'blocks.1.res_blocks.0.branch2.norm_b.1.mul', 'blocks.1.res_blocks.0.branch2.act_b.sigmoid', 'blocks.1.res_blocks.0.branch2.act_b.mul', 'blocks.1.res_blocks.0.branch2.conv_c', 'blocks.1.res_blocks.0.branch2.norm_c', 'blocks.1.res_blocks.0.add', 'blocks.1.res_blocks.0.activation', 'blocks.1.res_blocks.1.branch2.conv_a', 'blocks.1.res_blocks.1.branch2.norm_a', 'blocks.1.res_blocks.1.branch2.act_a', 'blocks.1.res_block

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


torch.Size([1, 1, 2, 2, 400])